In [10]:
import numpy as np
import codecs
import re

In [47]:
#Prepare training data
fname = './president/corpus.tache1.learn.utf8'
f=codecs.open(fname, 'r','utf-8') # pour régler le codage
txts=f.readlines()
nblignes=len(txts)

#cptM= 0
#cptC=0
labels = [] # M:0, C:1
trainData = []
for t in txts:
    label = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",t)
    txt = re.sub(r"<[0-9]*:[0-9]*:.> (.*)","\\1",t)
    
    trainData.append(txt)   
    
    if label[0]=="C":
        #cptC+=1
        labels.append(1)
    else:
        #cptM +=1
        labels.append(0)
    
f.close()

In [48]:

Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(trainData,labels,test_size=0.3,random_state=1)

In [49]:
#print(trainData[0])

Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.



In [53]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import nltk


Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

#Tfidf_vect = TfidfVectorizer(max_features=5000)
list_stopwords = stopwords.words('french')
Tfidf_vect = TfidfVectorizer(stop_words=list_stopwords,ngram_range=(1,2))
Tfidf_vect.fit_transform(trainData)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

SVM Accuracy Score ->  89.01532745006968


In [ ]:
#without Hyper-parameter Tuning

# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [54]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
  
# defining parameter range 
param_grid = [{'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf'],
              'class_weight': ['balanced']},
              
              {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf'],
              'class_weight': [{1: 6.6}]},
              
              {'kernel': ['linear'], 
              'C': [1, 10, 100, 1000],
              'class_weight': ['balanced']},
             
             {'kernel': ['linear'], 
              'C': [1, 10, 100, 1000],
              'class_weight': [{1: 6.6}]}]
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1',return_train_score=True) 
  
# fitting the model for grid search 
grid.fit(Train_X_Tfidf, Train_Y) 

Fitting 5 folds for each of 58 candidates, totalling 290 fits
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.878, total= 8.5min
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.5min remaining:    0.0s


[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.872, total= 8.6min
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 17.1min remaining:    0.0s


[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.875, total= 8.6min
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.875, total= 8.7min
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.872, total= 8.8min
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf .............
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf, score=0.877, total= 7.9min
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf .............
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf, score=0.873, total= 7.8min
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf .............
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf, score=0.877, total= 7.9min
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf .............
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf, score

KeyboardInterrupt: 

In [ ]:
print(gs_clf_movie.best_score_)

# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

In [ ]:
from sklearn.metrics import f1_score

grid_predictions = grid.predict(Test_X) 
f1=f1_score(Test_Y,grid_predictions) 

# print classification report 
print(classification_report(Test_Y, grid_predictions)) 

In [ ]:
fname = './president/corpus.tache1.test.utf8'
s=codecs.open(fname, 'r','utf-8') # pour régler le codage
lignes_test=s.readlines()
print(lignes_test[0])

nblignes = len(lignes_test)
test = []
for txt in lignes_test:
    txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
    test.append(txt)
    
#test=np.array(test)
prediction = grid.predict(test)

print(prediction)

In [ ]:
#Post-processing

for i in range(1,len(prediction)-1):
    if prediction[i] != prediction[i-1] and prediction[i] != prediction[i+1]:
        prediction[i] = prediction[i-1]


res_file = "./SVM/predictionPolitique.txt"
f=open(res_file,'w')
for p in prediction:
    f.write(str(int(p))+"\n")
f.close()

print("done")

In [ ]:
#############################################################################################################################################

In [ ]:
import os.path
def readAFile(path):
    text_file = open(path, "r")
    lines = text_file.read()
    return lines
    
path = "./movies/movies1000/"

alltxts_movie = [] # init vide
labs_movie = []
cpt = 0
for cl in os.listdir(path): # parcours des fichiers d'un répertoire
    print(cl)
    for f in os.listdir(path+cl):
        txt = readAFile(path+cl+'/'+f)
        alltxts_movie.append(txt)
        labs_movie.append(cpt) # 0 pour neg et 1 pour pos
    cpt += 1
    #print(len(alltxts_movie))
#print(len(alltxts_movie))

In [ ]:
Train_X_movie, Test_X_movie, Train_Y_movie, Test_Y_movie = model_selection.train_test_split(alltxts_movie,labs_movie,test_size=0.3,random_state=1)

In [ ]:
Encoder_movie = LabelEncoder()
Train_Y_movie = Encoder_movie.fit_transform(Train_Y_movie)
Test_Y_movie = Encoder_movie.fit_transform(Test_Y_movie)

list_stopwords_english = stopwords.words('english')
Tfidf_vect_movie = TfidfVectorizer(stop_words=list_stopwords_english,ngram_range=(1,2))
Tfidf_vect_movie.fit_transform(alltxts_movie)
Train_X_Tfidf_movie = Tfidf_vect_movie.transform(Train_X_movie)
Test_X_Tfidf_movie = Tfidf_vect_movie.transform(Test_X_movie)

In [ ]:
# defining parameter range 
param_grid_movie = [{'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf'],
              'class_weight': ['balanced']},              
              {'kernel': ['linear'], 
              'C': [1, 10, 100, 1000],
              'class_weight': ['balanced']},]
  
grid_movie = GridSearchCV(SVC(), param_grid_movie, refit = True, verbose = 3,n_jobs=-1,scoring='f1',return_train_score=True) 
  
# fitting the model for grid search 
grid_movie.fit(Train_X_Tfidf_movie, Train_Y_movie) 

In [ ]:
print(gs_clf_movie.best_score_)

# print best parameter after tuning 
print(grid_movie.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid_movie.best_estimator_) 

In [ ]:
grid_predictions_movie = grid_movie.predict(Test_X_movie) 
f1_movie=f1_score(Test_Y_movie,grid_predictions_movie) 

# print classification report 
print(classification_report(Test_Y_movie, grid_predictions_movie)) 

In [ ]:
#Recuperation 
#Prediction du fichier test avec une ligne = 1 avis
f=open("testSentiment.txt",'r',encoding="utf8")
test_movies = f.readlines()
#print(len(test_movies))
f.close()

#test = np.array(test_movies)
prediction_test_movie = grid_movie.predict(test_movies)

#print(prediction)
#print(len(prediction))
sortie = "./SVM/predictionMovies.txt"
f=open(sortie,'w')
for p in prediction_test_movie:
    f.write(str(int(p))+"\n")
f.close()
